In [12]:
import numpy as np #ref
import os
from os import path as op
import dask #ref
import string
import re
from subprocess import Popen
import xarray as xr #ref
import metpy #ref
import matplotlib.pyplot as plt #ref
import cartopy.crs as ccrs #ref
import cartopy.feature as cfeature
from pyresample import image, geometry #ref
import seaborn as sns #ref
import Download_script.py
sns.set(style="darkgrid")
%matplotlib inline

#sys.path.append('')


ModuleNotFoundError: No module named 'Download_script'

In [2]:
path = "G16_G17_url_list/"
dirs = os.listdir( path )

In [14]:
def createNumpy(ncFile, pathIn, pathOut):
    myFile = xarray.open_dataset(op.join(pathIn,ncFile))
    dat = myFile.metpy.parse_cf('Rad')
    geos = dat.metpy.cartopy_crs

    cartopy_extent_goes = geos.x_limits + geos.y_limits
    pyresample_extent_goes = (cartopy_extent_goes[0],
                                cartopy_extent_goes[2],
                                cartopy_extent_goes[1],
                                cartopy_extent_goes[3])
    goes_params = geos.proj4_params
    rad = dat.data
    
    def normIm(im,gamma=1.0,reverse=False):
        nim = ((im-np.nanmin(im))*(np.nanmax(im)-np.nanmin(im))**(-1))
        if reverse:#want clouds to be white
            nim = (1.0-nim**(gamma))
        return nim
    
    def goes_2_roi(loaded_goes, 
               target_extent,
               target_rows,
               target_cols,
               cartopy_target_proj,
               data_key='Rad',
               radius_of_influence=50000):
        """Function that goes from loaded GOES data to data resampled in a projection for an extent"""
        dat = loaded_goes.metpy.parse_cf('Rad')
        geos_crs = dat.metpy.cartopy_crs
        cartopy_source_extent = geos_crs.x_limits + geos_crs.y_limits
        pyresample_source_extent = (cartopy_source_extent[0],
                                    cartopy_source_extent[2],
                                    cartopy_source_extent[1],
                                    cartopy_source_extent[3])
        rad = dat.data
        source_area = geometry.AreaDefinition('GOES-1X', 'Full Disk','GOES-1X', 
                                              geos_crs.proj4_params,
                                              rad.shape[1], rad.shape[0],
                                              pyresample_source_extent)
        area_target_def = geometry.AreaDefinition('areaTest', 'Target Region', 'areaTest',
                                            cartopy_target_proj.proj4_params,
                                            target_rows, target_cols,
                                            target_extent)
        geos_con_nn = image.ImageContainerNearest(rad, 
                                                source_area, 
                                                radius_of_influence=radius_of_influence)

        # Here we are using pyresample for the remapping
        area_proj_con_nn = geos_con_nn.resample(area_mc_def)
        return area_proj_con_nn.image_data
        
    def cartopy_pyresample_toggle_extent(input_extent):
        return np.array(input_extent)[np.array([0,2,1,3])]

    def trasform_cartopy_extent(source_extent,source_proj, target_proj):
        target_extent = target_proj.transform_points(source_proj, 
                                                     np.array(source_extent[:2]),
                                                     np.array(source_extent[2:])).ravel()
        # target_extent in 3D, must be in 2D
        return cartopy_pyresample_toggle_extent(np.array(target_extent)[np.array([0,1,3,4])])
    pc = ccrs.PlateCarree()
    mc = ccrs.Mercator()

    # Convert extent from pc to mc (both cylindrical projections)
    extent_pc = [-109.59326, -102.40674, 8.94659, -8.94656]
    
    target_extent_mc_cartopy = trasform_cartopy_extent(extent_pc, pc, mc)
    target_extent_mc_pyresample = cartopy_pyresample_toggle_extent(target_extent_mc_cartopy)
    
    roi_rads = goes_2_roi(myFile,
               target_extent_mc_pyresample,
               500,
               200,
               mc)
    ####
    full_filename = op.join(pathOut,ncFile[:-3])
    np.save(full_filename,roi_rads)
    return

In [39]:
from pathlib import Path
newPath = '../../../../../sharedData/scratch/'

for txtFile in dirs:
    txtFileName = txtFile[:-4]
    subDir = op.join(newPath, 'npOut')
    dirName = op.join(subDir, txtFileName)
    cmdList = [ 'mkdir -p '+dirName]
    print (cmdList)

    pMkdir = Popen(cmdList,shell = True)
    pMkdir.communicate()
    
    data = Path ( newPath )
    res = [f for f in os.listdir(data) if re.search('G17_s2019108',f)]
    
    for nc in res:
        outPath = op.join(newPath,dirName)
        createNumpy(nc,newPath,dirName)

['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_08-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_09-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_10-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_11-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_12-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_13-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_14-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_15-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_16-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_17-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G16_data_04_18-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G17_data_04_08-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G17_data_04_09-19']
['mkdir -p ../../../../../sharedData/scratch/npOut/G17_data_04_10-19']
['mkdi

In [13]:
data.exists()

True

In [24]:
files = data.glob("*G17_s201910819")

In [22]:
list(files)

[]

In [38]:
print(len(res))

572
